In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time
import json
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

class FullScaleConfig:

    DATA_DIRS = {
        'train': './dataset/train',   
        'val': './dataset/val',        
        'test': './dataset/test'       
    }

    TRAIN_SAMPLES = None      # None表示使用全部
    VAL_SAMPLES = None        
    TEST_SAMPLES = None       
    

    INPUT_DIM = 255
    OUTPUT_DIM = 50
    HIDDEN_DIMS = [512, 256, 128, 64]  
    DROPOUT_RATE = 0.35       

    BATCH_SIZE = 64          
    EPOCHS = 300              
    LEARNING_RATE = 0.0015    
    WEIGHT_DECAY = 1e-4       

    EARLY_STOP_PATIENCE = 50  

    OPTIMIZER = 'AdamW'
    USE_GRADIENT_CLIP = True
    GRADIENT_CLIP_VAL = 1.0   

    NUM_WORKERS = 0           
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    RUN_ID = datetime.now().strftime("%Y%m%d_%H%M%S")
    SAVE_DIR = Path(f'./full_scale_results/run_{RUN_ID}')
    SAVE_DIR.mkdir(parents=True, exist_ok=True)
    
    def __init__(self):
        print("=" * 80)
        print("完整规模训练配置 - 90k训练 + 15k验证 + 5k测试")
        print("=" * 80)
        print(f"数据规模: 完整数据集")
        print(f"模型架构: {self.INPUT_DIM} → {self.HIDDEN_DIMS} → {self.OUTPUT_DIM}")
        print(f" 超参数: LR={self.LEARNING_RATE}, BS={self.BATCH_SIZE}, Dropout={self.DROPOUT_RATE}")
        print(f"最大周期: {self.EPOCHS}, 早停耐心={self.EARLY_STOP_PATIENCE}")
        print(f"设备: {self.DEVICE}")
        
        if torch.cuda.is_available():
            gpu_name = torch.cuda.get_device_name(0)
            print(f"GPU: {gpu_name}")

class SmartBeamDataset(Dataset):

    def __init__(self, data_dir, max_samples=None, shuffle=True):
        self.data_dir = Path(data_dir)
        self.max_samples = max_samples
        self.shuffle = shuffle
        self.samples = []
        
        print(f"\n加载数据从: {data_dir}")
        print(f"   目标样本数: {max_samples if max_samples else '全部'}")
        
        start_time = time.time()
        self._load_and_cache_data()
        load_time = time.time() - start_time
        
        print(f"加载完成: {len(self.samples):,} 个样本")
    
    def _load_and_cache_data(self):

        batch_files = sorted(self.data_dir.glob('batch_*.pkl'))
        
        if not batch_files:
            raise FileNotFoundError(f"未找到批次文件: {self.data_dir}")
        
        print(f"   发现 {len(batch_files)} 个批次文件")
        
        loaded_samples = 0
        for file_idx, batch_file in enumerate(batch_files):

            if self.max_samples and loaded_samples >= self.max_samples:
                break
            

            with open(batch_file, 'rb') as f:
                batch_data = pickle.load(f)

            remaining = self.max_samples - loaded_samples if self.max_samples else len(batch_data)
            samples_to_add = min(len(batch_data), remaining)

            self.samples.extend(batch_data[:samples_to_add])
            loaded_samples += samples_to_add

            if (file_idx + 1) % 5 == 0 or (file_idx + 1) == len(batch_files):
                print(f"   [{file_idx+1:03d}/{len(batch_files)}] 已加载 {loaded_samples:,} 样本")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]

        freq = torch.FloatTensor(sample['input']['frequencies'][:5])
        mode_shapes = torch.FloatTensor(sample['input']['mode_shapes'][:5])
        x = torch.cat([freq, mode_shapes.flatten()])
        y = torch.FloatTensor(sample['output']['binary_labels'])
        
        return x, y

class FullScaleFCN(nn.Module):
    
    def __init__(self, input_dim, output_dim, hidden_dims, dropout_rate=0.3):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        for i, h_dim in enumerate(hidden_dims):
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.BatchNorm1d(h_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            prev_dim = h_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.01)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        return self.network(x)

class SimpleEarlyStopping:
    def __init__(self, patience=20, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
    
    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            improved = True
        else:
            self.counter += 1
            improved = False
            
            if self.counter >= self.patience:
                self.early_stop = True
        
        return self.early_stop, improved

def compute_accuracy(predictions, targets, threshold=0.5):
    probs = torch.sigmoid(predictions)
    binary_preds = (probs > threshold).float()
    correct = (binary_preds == targets).float()
    return correct.mean().item()


def train_epoch(model, loader, criterion, optimizer, device, config):

    model.train()
    total_loss = 0.0
    total_acc = 0.0
    total_samples = 0
    
    for batch_idx, (batch_x, batch_y) in enumerate(loader):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_size = batch_x.size(0)
        
        optimizer.zero_grad()
        predictions = model(batch_x)
        loss = criterion(predictions, batch_y)
        
        loss.backward()
        
        if config.USE_GRADIENT_CLIP:
            torch.nn.utils.clip_grad_norm_(model.parameters(), config.GRADIENT_CLIP_VAL)
        
        optimizer.step()

        total_loss += loss.item() * batch_size
        total_acc += compute_accuracy(predictions, batch_y) * batch_size
        total_samples += batch_size

        if (batch_idx + 1) % 50 == 0:
            avg_loss = total_loss / total_samples
            avg_acc = total_acc / total_samples
            print(f'    批次 {batch_idx+1:04d}/{len(loader)} | 损失: {avg_loss:.4f} | 准确率: {avg_acc:.3f}')
    
    avg_loss = total_loss / total_samples
    avg_acc = total_acc / total_samples
    return avg_loss, avg_acc

def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    total_samples = 0
    
    with torch.no_grad():
        for batch_x, batch_y in loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            batch_size = batch_x.size(0)
            
            predictions = model(batch_x)
            loss = criterion(predictions, batch_y)
            
            total_loss += loss.item() * batch_size
            total_acc += compute_accuracy(predictions, batch_y) * batch_size
            total_samples += batch_size

    avg_loss = total_loss / total_samples
    avg_acc = total_acc / total_samples
    return avg_loss, avg_acc

def test_model(model, config):
    print("\n" + "=" * 80)
    print("阶段4: 在测试集上进行最终评估")
    print("=" * 80)

    test_dataset = SmartBeamDataset(
        config.DATA_DIRS['test'],
        max_samples=config.TEST_SAMPLES,
        shuffle=False
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=False,
        num_workers=config.NUM_WORKERS
    )
    
    print(f" 测试集: {len(test_dataset):,} 个样本")
    
    pos_weight = torch.tensor([5.0]).to(config.DEVICE)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    test_loss, test_acc = validate(model, test_loader, criterion, config.DEVICE)
    
    model.eval()
    correct = 0
    total = 0
    
    damage_correct = 0
    damage_total = 0
    
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(config.DEVICE), y.to(config.DEVICE)
            outputs = model(x)
            preds = (torch.sigmoid(outputs) > 0.5).float()
    
            correct += (preds == y).sum().item()
            total += y.numel()
    
            damage_mask = (y == 1)
            damage_correct += (preds[damage_mask] == 1).sum().item()
            damage_total += damage_mask.sum().item()

    accuracy = correct / total
    damage_recall = damage_correct / damage_total if damage_total > 0 else 0.0

    
    print("Accuracy:", accuracy)
    print("Damage Recall:", damage_recall)

    return test_loss, test_acc, test_dataset

def main():
    config = FullScaleConfig()

    config_dict = {k: v for k, v in config.__dict__.items() 
                  if not k.startswith('_') and not callable(v)}
    with open(config.SAVE_DIR / 'training_config.json', 'w') as f:
        json.dump(config_dict, f, indent=2, default=str)
    
    print("\n" + "=" * 80)
    print("阶段1: 加载数据集")
    print("=" * 80)

    train_dataset = SmartBeamDataset(
        config.DATA_DIRS['train'],
        max_samples=config.TRAIN_SAMPLES,
        shuffle=True
    )
    
    val_dataset = SmartBeamDataset(
        config.DATA_DIRS['val'],
        max_samples=config.VAL_SAMPLES,
        shuffle=False
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=True,
        num_workers=config.NUM_WORKERS
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=False,
        num_workers=config.NUM_WORKERS
    )
    
    print(f"\n数据集统计:")
    print(f"   训练集: {len(train_dataset):,} 个样本, {len(train_loader)} 个批次")
    print(f"   验证集: {len(val_dataset):,} 个样本, {len(val_loader)} 个批次")

    print("\n" + "=" * 80)
    print("阶段2: 初始化模型")
    print("=" * 80)
    
    model = FullScaleFCN(
        config.INPUT_DIM,
        config.OUTPUT_DIM,
        config.HIDDEN_DIMS,
        dropout_rate=config.DROPOUT_RATE
    ).to(config.DEVICE)

    total_params = sum(p.numel() for p in model.parameters())
    print(f" 模型参数: {total_params:,}")
    print(f"   模型大小: {total_params * 4 / 1024**2:.2f} MB")

    pos_weight = torch.tensor([5.0]).to(config.DEVICE)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    
    if config.OPTIMIZER == 'AdamW':
        optimizer = optim.AdamW(
            model.parameters(),
            lr=config.LEARNING_RATE,
            weight_decay=config.WEIGHT_DECAY
        )
    else:
        optimizer = optim.Adam(
            model.parameters(),
            lr=config.LEARNING_RATE
        )

    early_stopping = SimpleEarlyStopping(
        patience=config.EARLY_STOP_PATIENCE
    )

    history = {
        'epoch': [],
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    best_val_acc = 0.0
    best_val_loss = float('inf')
    best_epoch = 0
    training_start_time = time.time()
    
    print("\n" + "=" * 80)
    print("阶段3: 开始完整规模训练")
    print("=" * 80)
    print(f"开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    for epoch in range(1, config.EPOCHS + 1):
        epoch_start_time = time.time()
        
        print(f"\nEpoch {epoch:03d}/{config.EPOCHS}")
        print("-" * 50)

        print("训练阶段:")
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, config.DEVICE, config
        )

        print("验证阶段:")
        val_loss, val_acc = validate(
            model, val_loader, criterion, config.DEVICE
        )

        epoch_time = time.time() - epoch_start_time
        total_time = time.time() - training_start_time

        history['epoch'].append(epoch)
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(f"\n📊 Epoch {epoch:03d} 总结:")
        print(f"   训练损失: {train_loss:.4f} | 训练准确率: {train_acc:.3f}")
        print(f"   验证损失: {val_loss:.4f} | 验证准确率: {val_acc:.3f}")
        print(f"   周期时间: {epoch_time:.1f}s | 累计时间: {total_time/60:.1f}分钟")

        early_stop, improved = early_stopping(val_loss)

        if improved or val_acc > best_val_acc:
            best_val_acc = max(val_acc, best_val_acc)
            best_val_loss = val_loss
            best_epoch = epoch
            
            best_checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'train_acc': train_acc,
                'val_loss': val_loss,
                'val_acc': val_acc,
                'history': history
            }
            
            torch.save(best_checkpoint, config.SAVE_DIR / 'best_model.pth')
            print(f"  保存最佳模型 @ Epoch {epoch}: 验证准确率 = {val_acc:.3f}")

        if epoch % 10 == 0:
            checkpoint_path = config.SAVE_DIR / f'checkpoint_epoch_{epoch:03d}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'val_acc': val_acc
            }, checkpoint_path)
            print(f"  检查点已保存: {checkpoint_path}")

        if early_stop:
            print(f"\n{'='*60}")
            print("训练完成 - 早停触发")
            print(f"最佳模型 @ Epoch {best_epoch}: 验证准确率 = {best_val_acc:.3f}")
            print(f"总训练时间: {total_time/60:.1f}分钟")
            print(f"{'='*60}")
            break

    if not early_stopping.early_stop:
        print(f"\n{'='*60}")
        print("训练完成 - 达到最大周期数")
        print(f"最佳模型 @ Epoch {best_epoch}: 验证准确率 = {best_val_acc:.3f}")
        print(f"总训练时间: {total_time/60:.1f}分钟")
        print(f"{'='*60}")

    print("\n" + "=" * 80)
    print("阶段4: 在测试集上评估最佳模型")
    print("=" * 80)

    best_checkpoint = torch.load(config.SAVE_DIR / 'best_model.pth')
    model.load_state_dict(best_checkpoint['model_state_dict'])

    test_loss, test_acc, test_dataset = test_model(model, config)
    
    print(f"\n测试集结果:")
    print(f"   测试损失: {test_loss:.4f}")
    print(f"   测试准确率: {test_acc:.3f}")

    print("\n" + "=" * 80)
    print("保存最终结果")
    print("=" * 80)

    torch.save(model.state_dict(), config.SAVE_DIR / 'final_model.pth')

    np.save(config.SAVE_DIR / 'training_history.npy', history)

    test_results = {
        'best_epoch': best_epoch,
        'best_val_loss': float(best_val_loss),
        'best_val_acc': float(best_val_acc),
        'test_loss': float(test_loss),
        'test_acc': float(test_acc),
        'total_epochs': len(history['epoch']),
        'total_training_time_minutes': total_time / 60,
        'training_finished': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    with open(config.SAVE_DIR / 'test_results.json', 'w') as f:
        json.dump(test_results, f, indent=2)

    plot_training_curves(history, config.SAVE_DIR, test_results)
    
    print(f"\n🎉 训练完成!")
    print(f" 结果保存在: {config.SAVE_DIR.absolute()}")
    print(f" 最佳验证准确率: {best_val_acc:.3%} @ Epoch {best_epoch}")
    print(f"测试集准确率: {test_acc:.3%}")

    medium_val_acc = 0.824  
    print(f"\n 性能对比:")
    print(f"   中等规模 (10k训练) 验证准确率: {medium_val_acc:.1%}")
    print(f"   完整规模 (90k训练) 验证准确率: {best_val_acc:.1%}")
    print(f"   完整规模 (90k训练) 测试准确率: {test_acc:.1%}")
    print(f"   验证集提升: {best_val_acc*100 - medium_val_acc*100:.1f}%")
    print(f"   测试集最终结果: {test_acc*100:.1f}%")
    
    return model, history, test_results

def plot_training_curves(history, save_dir, test_results):
    epochs = history['epoch']
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    fig.suptitle(f'Full data training (90k train + 15k val + 5k test)', fontsize=16, y=1.02)
    
    ax = axes[0, 0]
    ax.plot(epochs, history['train_loss'], 'b-', linewidth=2, label='Training loss')
    ax.plot(epochs, history['val_loss'], 'r-', linewidth=2, label='Verification loss')
    ax.axvline(x=test_results['best_epoch'], color='g', linestyle='--', alpha=0.5, label='Best model')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title('Training and validation losses')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    ax = axes[0, 1]
    ax.plot(epochs, history['train_acc'], 'b-', linewidth=2, label='Training accuracy')
    ax.plot(epochs, history['val_acc'], 'r-', linewidth=2, label='Verify the accuracy rate')
    ax.axvline(x=test_results['best_epoch'], color='g', linestyle='--', alpha=0.5)
    ax.axhline(y=test_results['best_val_acc'], color='r', linestyle=':', alpha=0.5, 
               label=f'最佳验证: {test_results["best_val_acc"]:.2%}')
    ax.axhline(y=test_results['test_acc'], color='purple', linestyle='-.', alpha=0.5,
               label=f'测试准确率: {test_results["test_acc"]:.2%}')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy rate')
    ax.set_title('Train and verify the accuracy')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim([0.5, 1.0])
    
    ax = axes[1, 0]
    scatter = ax.scatter(history['val_loss'], history['val_acc'], 
                        c=history['epoch'], cmap='viridis', 
                        s=30, alpha=0.7)
    ax.set_xlabel('Verification loss')
    ax.set_ylabel('Verify the accuracy rate')
    ax.set_title('Loss vs. accuracy relationship')
    ax.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax, label='Epoch')
    
    ax = axes[1, 1]
    ax.axis('off')
    
    final_train_acc = history['train_acc'][-1] if history['train_acc'] else 0
    final_val_acc = history['val_acc'][-1] if history['val_acc'] else 0
    
    summary_text = (
        f"训练总结\n"
        f"========\n"
        f"最佳周期: {test_results['best_epoch']}\n"
        f"最佳验证准确率: {test_results['best_val_acc']:.3%}\n"
        f"测试准确率: {test_results['test_acc']:.3%}\n"
        f"最终训练准确率: {final_train_acc:.3%}\n"
        f"最终验证准确率: {final_val_acc:.3%}\n"
        f"总训练周期: {len(epochs)}\n"
        f"训练时间: {test_results['total_training_time_minutes']:.1f}分钟\n"
        f"数据规模: 90k训练 + 15k验证 + 5k测试\n"
        f"批次大小: 128\n"
        f"学习率: 0.0005\n"
    )
    
    ax.text(0.1, 0.9, summary_text, transform=ax.transAxes,
            fontsize=10, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plot_path = save_dir / 'training_curves.png'
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    plt.close(fig)
    
    print(f"训练曲线已保存: {plot_path}")

if __name__ == "__main__":
    print("=" * 80)
    print("梁结构损伤识别 - 完整规模训练")
    print("=" * 80)
    
    try:

        print("\n系统检查...")
        print(f"PyTorch版本: {torch.__version__}")
        print(f"CUDA可用: {torch.cuda.is_available()}")
        
        if torch.cuda.is_available():
            print(f"GPU: {torch.cuda.get_device_name(0)}")
        else:
            print("警告: 未检测到GPU，将使用CPU训练")
        

        print("\n检查数据路径...")
        for data_type, path in FullScaleConfig.DATA_DIRS.items():
            data_path = Path(path)
            if data_path.exists():
                pkl_files = list(data_path.glob('batch_*.pkl'))
                print(f"✓ {data_type}: {len(pkl_files)}个批次文件")
            else:
                print(f"✗ {data_type}: 路径不存在 {path}")
                raise FileNotFoundError(f"数据路径不存在: {path}")
        
        print("\n开始完整规模训练...")
        
        # 运行训练
        model, history, test_results = main()
        
        print("\n" + "=" * 80)
        print("🎉 训练成功完成!")
        print("=" * 80)
        
    except Exception as e:
        print(f"\n 训练过程中出错: {e}")
        import traceback
        traceback.print_exc()
        
        # 保存错误信息
        error_dir = Path('./training_errors')
        error_dir.mkdir(exist_ok=True)
        error_file = error_dir / f'error_{datetime.now().strftime("%Y%m%d_%H%M%S")}.txt'
        
        with open(error_file, 'w') as f:
            f.write(f"训练错误: {e}\n")
            f.write(f"时间: {datetime.now()}\n")
            f.write("\nTraceback:\n")
            f.write(traceback.format_exc())
        
        print(f"错误详情已保存: {error_file}")